In [ ]:
from qiskit import *
import math
import numpy as np
import hashlib
import hmac
import time
import json
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import Padding

In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
# simulator = Aer.get_backend('qasm_simulator')
# simulator.num_qubits = 36
# dir(simulator)
# simulator.set_option("",36)
Aer.get_backend('asm_simulator')

In [ ]:
class QChannel():
    def __init__(self, numberOfCommunicators):
        # size of INFO BITS
        self.n = 3
        
        # delta parameter
        self.delta = 1/8

        # N
        self.N = math.ceil(8*self.n*(1+self.delta))

        # probability of not measuring by communicators
        self.c = 0.7

        self.numberOfCommunicators = numberOfCommunicators

    def performAlgorithm(self):

        # quantum channel
        circuit = QuantumCircuit(self.N, self.N)

        # encoding of initial state
        # 0 = |+> and 1 = |->
        initialStateDecision = np.random.randint(2, size=self.N)

        print("initialStateDecision")
        print(initialStateDecision)

        for i in range(len(initialStateDecision)):
            if initialStateDecision[i]:
                circuit.x(i)
            circuit.h(i)


        # decisions by each communicator

        allMeasureDecisions = []*self.numberOfCommunicators

        for i in range(self.numberOfCommunicators):
            allMeasureDecisions.append(list(np.random.choice([0, 1], self.N, p=[self.c, 1-self.c])))
        
        print("allMeasureDecisions")
        for amd in allMeasureDecisions:
            print(amd)


        for i in range(self.numberOfCommunicators):
            for j in range(self.N):
                if allMeasureDecisions[i][j]:
                    circuit.measure(j, j)

        # decisions of measure basis by channel
        # 0 = Z and 1 = H

        channelMeasurementDecision = np.random.randint(2, size=self.N)
        
        print("channelMeasurementDecision")
        print(channelMeasurementDecision)

        for d, idx in enumerate(initialStateDecision):
            if d:
                circuit.h(idx)
            circuit.measure(idx, idx)

        # (p0) None of Bi measured the travel qubit
        # (p1) One of Bi measured the travel qubit but the other did not
        # (p2) More than one of Bi measured the travel qubit.

        cases = [0]*self.N
        

        for i in range(self.numberOfCommunicators):
            for j in range(self.N):
                if allMeasureDecisions[i][j] and cases[j] <2:
                    cases[j] += 1



        print("cases")
        print(cases)
        
        ctrl_A = []
        ctrl_B = []
        sift = []

        for i in range(len(cases)):
            if cases[i] == 0:
                ctrl_A.append(i)
            elif cases[i] == 1:
                ctrl_B.append(i)
            else:
                sift.append(i)

        print(f'{ctrl_A=}')
        print(f'{ctrl_B=}')
        print(f'{sift=}')


        # decisions of measure basis by channel
        # 0 = Z and 1 = H

        finalMeasurementBasis = np.random.randint(2, size=self.N)

        for i in range(self.N):
            if finalMeasurementBasis[i]:
                circuit.h(i)

        print("finalMeasurementBasis")
        print(finalMeasurementBasis)

        result= execute(circuit, backend=simulator, shots=1).result()
        count = result.get_counts(circuit)
        value = list(count.keys())[0][::-1]

        print(value)
        

        print(circuit.draw())

In [ ]:
q = QChannel(3)
q.performAlgorithm()